In [3]:
# Setup: imports, paths, data load, and helpers for the visualizations
from pathlib import Path
import pandas as pd
import json
import datetime as dt
import plotly.express as px

# Find repository root from this notebook's working directory
_DEF_START = Path.cwd()

def find_root(start: Path = _DEF_START) -> Path:
    for p in [start] + list(start.parents):
        if (p / 'docs').exists() and (p / 'data_processed_ml').exists():
            return p
    return start

ROOT = find_root()
DOCS = ROOT / 'docs'
DOCS_DATA = DOCS / 'data'
DOCS_DATA.mkdir(parents=True, exist_ok=True)
OUT_ML = ROOT / 'data_processed_ml'
OUT_ML.mkdir(parents=True, exist_ok=True)

# Load latest tidy dataset produced by this pipeline
csv_candidates = sorted(OUT_ML.glob('educacion_regional_tidy_*.csv'), key=lambda p: p.stat().st_mtime, reverse=True)
if not csv_candidates:
    raise FileNotFoundError('No se encontró ningún archivo tidy en data_processed_ml/educacion_regional_tidy_*.csv')
latest_csv = csv_candidates[0]

df = pd.read_csv(latest_csv)
# Ensure numeric types
for col in ['ocupados_informales','ocupados_formales','ocupados_totales']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)
# Ensure tasa exists
if 'tasa_informal' not in df.columns:
    df['tasa_informal'] = (df['ocupados_informales'] / df['ocupados_totales']).fillna(0)

# Last available period
ultimo_trimestre = df['trimestre'].max()

# Plotly defaults and styling
px.defaults.width = None
px.defaults.height = None
px.defaults.template = 'plotly_white'

ECON_LAYOUT = dict(
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(family='Georgia, serif', color='#1e293b'),
    legend=dict(orientation='h', x=0.02, y=-0.18)
)

EDU_ORDER = ['Sin educación','Básica incompleta','Básica completa',
             'Media incompleta','Media completa','Superior incompleta','Superior completa']

# Helper to style axes in an Economist-like manner
def economist_axes(fig):
    fig.update_xaxes(showgrid=False, zeroline=False, linecolor='#cbd5e1')
    fig.update_yaxes(gridcolor='#e2e8f0', gridwidth=0.6, zeroline=False, linecolor='#cbd5e1')
    return fig

# Save JSON helper

def save_json(obj, path: Path):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(obj, f, ensure_ascii=False, indent=2)
    return str(path)

print('✔️ Setup OK')
print('Raíz:', ROOT)
print('CSV:', latest_csv.name)
print('Último período:', ultimo_trimestre)

✔️ Setup OK
Raíz: /Users/brunosanmartinnavarro/Documents/UACh/education-employment-analysis
CSV: educacion_regional_tidy_20250825_021407.csv
Último período: 2024Q4


# Ocupados por nivel educativo x sexo y región (Chile)

Este cuaderno construye una tabla maestra de ocupados formales e informales por nivel educativo, sexo y región, y genera varias visualizaciones estilo The Economist. También exporta JSONs y crea un segundo notebook de dashboard regional.

In [6]:
# 11) Vis 5: Mapa de burbujas (aproximado)
# Coordenadas aproximadas de centroides regionales (lat, lon)
REGION_COORDS = {
    'Tarapacá': (-20.2, -69.3),
    'Antofagasta': (-23.6, -69.4),
    'Atacama': (-27.4, -70.6),
    'Coquimbo': (-30.7, -71.2),
    'Valparaíso': (-33.0, -71.6),
    'O’Higgins': (-34.3, -70.8),
    'Maule': (-35.4, -71.7),
    'Ñuble': (-36.6, -72.1),
    'Biobío': (-37.2, -72.9),
    'La Araucanía': (-38.5, -72.6),
    'Los Ríos': (-39.8, -73.2),
    'Los Lagos': (-41.5, -73.0),
    'Aysén': (-46.0, -72.1),
    'Magallanes': (-53.1, -70.9),
    'Metropolitana': (-33.5, -70.7),
}
geo = (df[df['trimestre']==ultimo_trimestre]
       .groupby('region', as_index=False)
       .agg(tasa=('tasa_informal','mean'), ocupados=('ocupados_totales','sum')))
geo = geo[geo['region'].isin(REGION_COORDS.keys())].copy()
geo['lat'] = geo['region'].map(lambda r: REGION_COORDS[r][0])
geo['lon'] = geo['region'].map(lambda r: REGION_COORDS[r][1])
fig_geo = px.scatter_geo(geo, lat='lat', lon='lon', size='ocupados', color='tasa',
                         color_continuous_scale='Reds', scope='south america',
                         hover_name='region', projection='natural earth',
                         size_max=30, labels={'tasa':'Tasa informal'})
fig_geo.update_layout(**ECON_LAYOUT, title=f'Mapa (burbujas) — Tasa informal y tamaño de ocupados ({ultimo_trimestre})')
fig_geo.show()

In [7]:
# 13) Persistencia de tabla tidy y diccionario de variables
OUT_ML = ROOT / 'data_processed_ml'
OUT_ML.mkdir(parents=True, exist_ok=True)

fname = f'educacion_regional_tidy_{dt.datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
path_csv = OUT_ML / fname

cols_order = ['trimestre','region','sexo','nivel_educativo', 'ocupados_informales','ocupados_formales','ocupados_totales','tasa_informal']
df[cols_order].to_csv(path_csv, index=False)

# Diccionario simple
dic = [
    {'col':'trimestre','desc':'Trimestre móvil (string, p.ej. 2024Q1)'},
    {'col':'region','desc':'Región administrativa de Chile'},
    {'col':'sexo','desc':'Hombres / Mujeres'},
    {'col':'nivel_educativo','desc':'Nivel educativo ordenado'},
    {'col':'ocupados_informales','desc':'Número de ocupados informales'},
    {'col':'ocupados_formales','desc':'Número de ocupados formales'},
    {'col':'ocupados_totales','desc':'Suma de formales e informales'},
    {'col':'tasa_informal','desc':'ocupados_informales / ocupados_totales'},
]
save_json({'archivo': str(path_csv), 'diccionario': dic}, OUT_ML / 'diccionario_educacion_regional.json')

print('✔️ Guardado maestro tidy en', path_csv)

✔️ Guardado maestro tidy en /Users/brunosanmartin/Documents/UACh/formalidad-informalidad-laboral-sector-chile/data_processed_ml/educacion_regional_tidy_20250825_021407.csv


## Próximos pasos
- Opcional: integrar heatmap y ranking en `docs/index.html` con selectores de período.
- Agregar control para filtrar por sector económico si la fuente lo permite.
- Sustituir dataset sintético por fuente oficial (ENAE/INE) y verificar taxonomías.

In [4]:
# 15) Ocupados por nivel educativo y sexo — Región de Los Ríos (último período)
region_name = 'Los Ríos'
base_lr = df[(df['region']==region_name) & (df['trimestre']==ultimo_trimestre)].copy()
if base_lr.empty:
    raise ValueError(f'No hay datos para {region_name} en {ultimo_trimestre}')

# Asegurar totales e integer
base_lr['ocupados_totales'] = base_lr['ocupados_totales'].astype(int)

bar_highlight_color = "#d62828"
bar_neutral_color = "#0a9396"
bar_background_color = "#f7f2ea"
bar_grid_color = "#d9d3c5"
bar_text_color = "#1e1b18"
bar_font_family = "Georgia, serif"

sexo_color_map = {
    'Mujeres': bar_highlight_color,
    'Hombres': bar_neutral_color,
}

fig_lr = px.bar(
    base_lr,
    x='nivel_educativo',
    y='ocupados_totales',
    color='sexo',
    barmode='group',
    category_orders={'nivel_educativo': EDU_ORDER},
    color_discrete_map=sexo_color_map,
    text='ocupados_totales',
    hover_data={'ocupados_totales': ':,.0f'},
    labels={'nivel_educativo': 'Nivel educativo', 'ocupados_totales': 'Ocupados (personas)', 'sexo': 'Sexo'}
)

fig_lr.update_traces(
    texttemplate='%{text:,.0f}',
    textposition='outside',
    marker_line=dict(color='#ffffff', width=1.2),
    opacity=1.0
)

fig_lr.update_traces(
    selector=dict(name='Mujeres'),
    marker_line=dict(color='#ffffff', width=1.4)
)

underline_y = 1.08

fig_lr.update_layout(
    template='plotly_white',
    title=dict(
        text=f"Ocupados por nivel educativo y sexo — {region_name} — Último período disponible",
        font=dict(family=bar_font_family, size=24, color=bar_text_color),
        x=0.05,
        y=0.95
    ),
    font=dict(family=bar_font_family, size=15, color=bar_text_color),
    plot_bgcolor=bar_background_color,
    paper_bgcolor=bar_background_color,
    margin=dict(l=120, r=80, t=120, b=120),
    hoverlabel=dict(bgcolor='#ffffff', font=dict(family=bar_font_family, color=bar_text_color)),
    width=1100,
    height=700,
    legend=dict(
        orientation='h',
        x=0.0,
        y=-0.2,
        font=dict(family=bar_font_family, size=12, color=bar_text_color),
        bgcolor='rgba(0,0,0,0)'
    ),
    shapes=[
        dict(
            type='line',
            x0=0.0,
            x1=1.0,
            y0=underline_y,
            y1=underline_y,
            xref='paper',
            yref='paper',
            line=dict(color=bar_highlight_color, width=4)
        )
    ]
)

fig_lr.update_traces(
    hovertemplate="<b>%{x}</b><br>Sexo: %{legendgroup}<br>Ocupados: %{y:,.0f}<extra></extra>"
)

fig_lr.update_xaxes(
    title=dict(text='Nivel educativo', font=dict(family=bar_font_family, size=14, color=bar_text_color)),
    tickfont=dict(family=bar_font_family, size=13, color=bar_text_color),
    showgrid=True,
    gridcolor=bar_grid_color,
    griddash='dot',
    zeroline=False
)

fig_lr.update_yaxes(
    title=dict(text='Ocupados (personas)', font=dict(family=bar_font_family, size=14, color=bar_text_color)),
    tickfont=dict(family=bar_font_family, size=13, color=bar_text_color),
    showgrid=True,
    gridcolor=bar_grid_color,
    griddash='dot',
    zeroline=False,
    tickformat=',d'
)

fig_lr.show()

# Export JSON para web (se mantiene el campo periodo para trazabilidad interna)
lr_records = (base_lr[['nivel_educativo','sexo','ocupados_totales']]
              .assign(periodo=ultimo_trimestre)
              .to_dict(orient='records'))
save_json({'region': region_name, 'periodo': ultimo_trimestre, 'records': lr_records, 'units': 'persons'},
          DOCS_DATA / 'ocupados_edu_sexo_los_rios.json')
          
print('✔️ JSON exportado:', DOCS_DATA / 'ocupados_edu_sexo_los_rios.json')

✔️ JSON exportado: /Users/brunosanmartinnavarro/Documents/UACh/education-employment-analysis/docs/data/ocupados_edu_sexo_los_rios.json


In [5]:
# 16) Ocupados informales por nivel educativo y sexo — Región de Los Ríos (último período)
region_name = 'Los Ríos'
base_lr_inf = df[(df['region']==region_name) & (df['trimestre']==ultimo_trimestre)].copy()
if base_lr_inf.empty:
    raise ValueError(f'No hay datos para {region_name} en {ultimo_trimestre}')

# Asegurar enteros
base_lr_inf['ocupados_informales'] = base_lr_inf['ocupados_informales'].fillna(0).astype(int)

bar_highlight_color = "#d62828"
bar_neutral_color = "#0a9396"
bar_background_color = "#f7f2ea"
bar_grid_color = "#d9d3c5"
bar_text_color = "#1e1b18"
bar_font_family = "Georgia, serif"

sexo_color_map = {
    'Mujeres': bar_highlight_color,
    'Hombres': bar_neutral_color,
}

fig_lr_inf = px.bar(
    base_lr_inf,
    x='nivel_educativo',
    y='ocupados_informales',
    color='sexo',
    barmode='group',
    category_orders={'nivel_educativo': EDU_ORDER},
    color_discrete_map=sexo_color_map,
    text='ocupados_informales',
    hover_data={'ocupados_informales': ':,.0f'},
    labels={'nivel_educativo': 'Nivel educativo', 'ocupados_informales': 'Ocupados informales (personas)', 'sexo': 'Sexo'}
)

fig_lr_inf.update_traces(
    texttemplate='%{text:,.0f}',
    textposition='outside',
    marker_line=dict(color='#ffffff', width=1.2),
    opacity=1.0
)

fig_lr_inf.update_traces(
    selector=dict(name='Mujeres'),
    marker_line=dict(color='#ffffff', width=1.4)
)

underline_y = 1.08

fig_lr_inf.update_layout(
    template='plotly_white',
    title=dict(
        text=f"Ocupados informales por nivel educativo y sexo — {region_name} — Último período disponible",
        font=dict(family=bar_font_family, size=24, color=bar_text_color),
        x=0.05,
        y=0.95
    ),
    font=dict(family=bar_font_family, size=15, color=bar_text_color),
    plot_bgcolor=bar_background_color,
    paper_bgcolor=bar_background_color,
    margin=dict(l=120, r=80, t=120, b=120),
    hoverlabel=dict(bgcolor='#ffffff', font=dict(family=bar_font_family, color=bar_text_color)),
    width=1100,
    height=700,
    legend=dict(
        orientation='h',
        x=0.0,
        y=-0.2,
        font=dict(family=bar_font_family, size=12, color=bar_text_color),
        bgcolor='rgba(0,0,0,0)'
    ),
    shapes=[
        dict(
            type='line',
            x0=0.0,
            x1=1.0,
            y0=underline_y,
            y1=underline_y,
            xref='paper',
            yref='paper',
            line=dict(color=bar_highlight_color, width=4)
        )
    ]
)

fig_lr_inf.update_traces(
    hovertemplate="<b>%{x}</b><br>Sexo: %{legendgroup}<br>Ocupados informales: %{y:,.0f}<extra></extra>"
)

fig_lr_inf.update_xaxes(
    title=dict(text='Nivel educativo', font=dict(family=bar_font_family, size=14, color=bar_text_color)),
    tickfont=dict(family=bar_font_family, size=13, color=bar_text_color),
    showgrid=True,
    gridcolor=bar_grid_color,
    griddash='dot',
    zeroline=False
)

fig_lr_inf.update_yaxes(
    title=dict(text='Ocupados informales (personas)', font=dict(family=bar_font_family, size=14, color=bar_text_color)),
    tickfont=dict(family=bar_font_family, size=13, color=bar_text_color),
    showgrid=True,
    gridcolor=bar_grid_color,
    griddash='dot',
    zeroline=False,
    tickformat=',d'
)

fig_lr_inf.show()

# Export JSON
inf_records = (base_lr_inf[['nivel_educativo','sexo','ocupados_informales']]
               .assign(periodo=ultimo_trimestre)
               .to_dict(orient='records'))
save_json({'region': region_name, 'periodo': ultimo_trimestre, 'records': inf_records, 'units': 'persons'},
          DOCS_DATA / 'ocupados_informales_edu_sexo_los_rios.json')
print('✔️ JSON exportado:', DOCS_DATA / 'ocupados_informales_edu_sexo_los_rios.json')

✔️ JSON exportado: /Users/brunosanmartinnavarro/Documents/UACh/education-employment-analysis/docs/data/ocupados_informales_edu_sexo_los_rios.json


In [ ]:
# 17) Ocupados formales por nivel educativo y sexo — Región de Los Ríos (último período)
base_lr_form = df[(df['region']=='Los Ríos') & (df['trimestre']==ultimo_trimestre)].copy()
if base_lr_form.empty:
    raise ValueError('No hay datos para Los Ríos en el último trimestre')

# Asegurar enteros
base_lr_form['ocupados_formales'] = base_lr_form['ocupados_formales'].fillna(0).astype(int)

bar_highlight_color = "#d62828"
bar_neutral_color = "#0a9396"
bar_background_color = "#f7f2ea"
bar_grid_color = "#d9d3c5"
bar_text_color = "#1e1b18"
bar_font_family = "Georgia, serif"

sexo_color_map = {
    'Mujeres': bar_highlight_color,
    'Hombres': bar_neutral_color,
}

fig_lr_form = px.bar(
    base_lr_form,
    x='ocupados_formales',
    y='nivel_educativo',
    color='sexo',
    orientation='h',
    barmode='group',
    category_orders={'nivel_educativo': EDU_ORDER[::-1]},
    color_discrete_map=sexo_color_map,
    text='ocupados_formales',
    hover_data={'ocupados_formales': ':,.0f'},
    labels={'nivel_educativo': 'Nivel educativo', 'ocupados_formales': 'Ocupados formales (personas)', 'sexo': 'Sexo'}
)

fig_lr_form.update_traces(
    texttemplate='%{text:,.0f}',
    textposition='outside',
    marker_line=dict(color='#ffffff', width=1.2),
    opacity=1.0
)

fig_lr_form.update_traces(
    selector=dict(name='Mujeres'),
    marker_line=dict(color='#ffffff', width=1.4)
)

underline_y = 1.08

fig_lr_form.update_layout(
    template='plotly_white',
    title=dict(
        text='Ocupados formales por nivel educativo y sexo — Los Ríos — Último período disponible',
        font=dict(family=bar_font_family, size=24, color=bar_text_color),
        x=0.05,
        y=0.95
    ),
    font=dict(family=bar_font_family, size=15, color=bar_text_color),
    plot_bgcolor=bar_background_color,
    paper_bgcolor=bar_background_color,
    margin=dict(l=160, r=120, t=120, b=120),
    hoverlabel=dict(bgcolor='#ffffff', font=dict(family=bar_font_family, color=bar_text_color)),
    width=1100,
    height=720,
    legend=dict(
        orientation='h',
        x=0.0,
        y=-0.2,
        font=dict(family=bar_font_family, size=12, color=bar_text_color),
        bgcolor='rgba(0,0,0,0)'
    ),
    shapes=[
        dict(
            type='line',
            x0=0.0,
            x1=1.0,
            y0=underline_y,
            y1=underline_y,
            xref='paper',
            yref='paper',
            line=dict(color=bar_highlight_color, width=4)
        )
    ]
)

fig_lr_form.update_traces(
    hovertemplate="<b>%{y}</b><br>Sexo: %{legendgroup}<br>Ocupados formales: %{x:,.0f}<extra></extra>"
)

fig_lr_form.update_xaxes(
    title=dict(text='Ocupados formales (personas)', font=dict(family=bar_font_family, size=14, color=bar_text_color)),
    tickfont=dict(family=bar_font_family, size=13, color=bar_text_color),
    showgrid=True,
    gridcolor=bar_grid_color,
    griddash='dot',
    zeroline=False,
    tickformat=',d'
)

fig_lr_form.update_yaxes(
    title=dict(text='Nivel educativo', font=dict(family=bar_font_family, size=14, color=bar_text_color)),
    tickfont=dict(family=bar_font_family, size=13, color=bar_text_color),
    showgrid=False,
    zeroline=False
)

fig_lr_form.show()

# Export JSON
form_records = (base_lr_form[['nivel_educativo','sexo','ocupados_formales']]
                .assign(periodo=ultimo_trimestre)
                .to_dict(orient='records'))
save_json({'region': 'Los Ríos', 'periodo': ultimo_trimestre, 'records': form_records, 'units': 'persons'},
          DOCS_DATA / 'ocupados_formales_edu_sexo_los_rios.json')
print('✔️ JSON exportado:', DOCS_DATA / 'ocupados_formales_edu_sexo_los_rios.json')

✔️ JSON exportado: /Users/brunosanmartinnavarro/Documents/UACh/education-employment-analysis/docs/data/ocupados_formales_edu_sexo_los_rios.json
